In [1]:
from datasets import load_dataset 
dataset = load_dataset("allenai/c4", "en", streaming=True)
#print(dataset)




Resolving data files:   0%|          | 0/1024 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1024 [00:00<?, ?it/s]

In [3]:
first_example = next(iter(dataset["train"]))
print(f"Keys: {first_example.keys()}")
print(first_example)

Keys: dict_keys(['text', 'timestamp', 'url'])
{'text': 'Beginners BBQ Class Taking Place in Missoula!\nDo you want to get better at making delicious BBQ? You will have the opportunity, put this on your calendar now. Thursday, September 22nd join World Class BBQ Champion, Tony Balay from Lonestar Smoke Rangers. He will be teaching a beginner level class for everyone who wants to get better with their culinary skills.\nHe will teach you everything you need to know to compete in a KCBS BBQ competition, including techniques, recipes, timelines, meat selection and trimming, plus smoker and fire information.\nThe cost to be in the class is $35 per person, and for spectators it is free. Included in the cost will be either a t-shirt or apron and you will be tasting samples of each meat that is prepared.', 'timestamp': '2019-04-25 12:57:54', 'url': 'https://klyq.com/beginners-bbq-class-taking-place-in-missoula/'}


In [5]:
# Split those data into files


In [6]:
from tqdm.auto import tqdm

ALREADY_PROCESSED = True

if not ALREADY_PROCESSED:
    text_data = []
    file_count = 0
    
    for sample in tqdm(dataset['train']):
        sample = sample['text'].replace('\n', '')
        sample = sample.lower()
        text_data.append(sample)
        if len(text_data) == 6_000:
            # once we git the 6K mark, save to file
            with open(f'c4_files/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
                fp.write('\n'.join(text_data))
            text_data = []
            file_count += 1
        if file_count >= 100:
            break

In [25]:
# Model definition
import torch
from transformers import BertConfig, BertForMaskedLM

device = torch.device('cuda')

# 加载预训练模型的配置（不加载权重）
config = BertConfig.from_pretrained("bert-base-uncased")
#print(config)

# fp32 by default
model = BertForMaskedLM(config)
model.to(device);

# If continue from previous epoch
#model.load_state_dict(torch.load('bert-trained/bert_ckpt_5.pth'))

#for name, param in model.named_parameters():
#    print(f"{name:50} shape: {str(param.shape):20} dtype: {param.dtype} param: {param}")

In [4]:
# Load tokenizer
# from transformers import BertTokenizer
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
print(tokenizer)

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)


In [9]:
# test tokenizer
text = "Hello, I love natural language processing!"
tokens = tokenizer.tokenize(text)
print("分词结果:", tokens)
input_ids = tokenizer.convert_tokens_to_ids(tokens)
print("Token IDs:", input_ids)

encoded = tokenizer.encode(text)
print("编码结果:", encoded)
print("解码:", tokenizer.decode(encoded))

分词结果: ['hello', ',', 'i', 'love', 'natural', 'language', 'processing', '!']
Token IDs: [7592, 1010, 1045, 2293, 3019, 2653, 6364, 999]
编码结果: [101, 7592, 1010, 1045, 2293, 3019, 2653, 6364, 999, 102]
解码: [CLS] hello, i love natural language processing! [SEP]


In [10]:
# Create the input pipeline and train

In [ ]:
INTERVAL=5
TARGET=25

#with open('c4_files/text_0.txt', 'r', encoding='utf-8') as fp:
#    lines = fp.read().split('\n') 

lines = []
for id in range(TARGET-INTERVAL, TARGET):
    file_path = f"c4_files/text_{id}.txt"
    print(file_path)
    with open(file_path, 'r', encoding='utf-8') as f:
        lines.extend(f.read().strip().split('\n'))
#print(lines[0])

#lines = [
#    "In the morning, I went to the school.",
#    "In the afternoon, I went to the school.",
#    "In the evening, I went to the school.",
#    "I love to eat bread for breakfast.",
#    "I love to eat butter for breakfast.",
#    "I love to eat noodle for breakfast.",
#    "The weather today is very good.",
#    "The weather today is very calm.",
#    "The weather today is very stormy.",
#    "he is a good man.",
#    "he is a gentle man.",
#    "he is a polite man.",
#    "She is a doctor.",
#    "She is a nurse.",
#    "She is a teacher.",
#]




In [45]:


tokenized_lines = tokenizer(lines, max_length=512, padding='max_length', truncation=True, return_tensors='pt')
#list(tokenized_lines.keys())

## Mask raw inputs
import torch

input_ids = tokenized_lines['input_ids']
attention_mask = tokenized_lines['attention_mask']

rand = torch.rand(input_ids.shape)
#print(rand)
# mask random 15% where token is not special tokens, id >= 999
mask_arr = (rand < 0.15) * (input_ids >= 999)
#print(mask_arr)
masked_input_ids = torch.where(mask_arr, tokenizer.mask_token_id, input_ids)
#print(masked_input_ids)

labels = input_ids.clone()
labels[attention_mask == 0] = -100
#print(labels)


# Define data loader
encodings = {'input_ids': masked_input_ids, 'attention_mask': attention_mask, 'labels': labels} 

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        # store encodings internally
        self.encodings = encodings

    def __len__(self):
        # return the number of samples
        return self.encodings['input_ids'].shape[0]

    def __getitem__(self, i):
        # return dictionary of input_ids, attention_mask, and labels for index i
        return {key: tensor[i] for key, tensor in self.encodings.items()}

dataset = Dataset(encodings) 
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True) 


# Training
from torch.optim import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=1e-4)

# Train with autocast bf16
from tqdm.auto import tqdm

epochs = 10

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        # mixed precision automatically
        with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
            outputs = model(input_ids, attention_mask=attention_mask,
                            labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item()) 

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [28]:
print(TARGET)
torch.save(model.state_dict(), f"bert-trained/bert_ckpt_{TARGET}.pth")

25


In [46]:
# Evaluate with fill mask 
from transformers import pipeline
import torch

# 创建fill-mask pipeline
fill_mask = pipeline(
    'fill-mask',
    model=model,  # 你的训练好的模型
    tokenizer=tokenizer,  # 对应的tokenizer
    device=0 if torch.cuda.is_available() else -1,  # GPU/CPU
    top_k=5  # 显示前5个预测结果
)

# 测试句子（包含[MASK] token）
test_sentences = [
    "In the [MASK], I went to the school.",
    "I love to eat [MASK] for breakfast.",
    "The weather today is very [MASK].",
    "[MASK] is a good man.",
    "She is a [MASK] doctor."
]

# 逐个预测
for sentence in test_sentences:
    print(f"\n输入: {sentence}")
    results = fill_mask(sentence)
    for i, result in enumerate(results):
        print(f"  {i+1}. {result['token_str']:15} (score: {result['score']:.4f})")

Device set to use cuda:0



输入: In the [MASK], I went to the school.
  1. to              (score: 0.2340)
  2. today           (score: 0.2187)
  3. afternoon       (score: 0.0073)
  4. a               (score: 0.0063)
  5. evening         (score: 0.0060)

输入: I love to eat [MASK] for breakfast.
  1. .               (score: 0.3847)
  2. bread           (score: 0.1910)
  3. no              (score: 0.0174)
  4. ##odle          (score: 0.0040)
  5. very            (score: 0.0023)

输入: The weather today is very [MASK].
  1. .               (score: 0.1605)
  2. the             (score: 0.0993)
  3. calm            (score: 0.0846)
  4. ##odle          (score: 0.0343)
  5. good            (score: 0.0075)

输入: [MASK] is a good man.
  1. she             (score: 0.9631)
  2. he              (score: 0.0017)
  3. the             (score: 0.0008)
  4. .               (score: 0.0003)
  5. to              (score: 0.0002)

输入: She is a [MASK] doctor.
  1. nurse           (score: 0.9528)
  2. .               (score: 0.0003)
  3. gen